# Calibrating particle numbers by photobleaching

*This document was generated from a Jupyter notebook.  You can download the notebook [here](photobleaching_hierarchical.ipynb).*

In [1]:
import warnings

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.special
import scipy.stats as st

# The MCMC Hammer
import emcee

# Import plotting tools
import bokeh.plotting as bp
import bokeh.io as bi
bi.output_notebook()

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Loading BokehJS ...

## Problem set-up

We seek to do the following experiment.  We subject a field of cells containing a fluorophore of interest to fluorescent light and take images as the fluorophores photobleach.  We wish to estimate a parameter $\nu$, which is the constant of proportionality between the number of fluorescent particles and the measured fluorescence intensity.

This was worked out in Nayak and Rutenberg, *Biophys. J.*, **101**, 2284-2293, 2011.  I present her another approach based on a Bayesian hierarchical model.

Let $I_i(t)$ by the measured integrated fluorescent intensity of cell $i$.  We assume that

\begin{align}
I_i(t) = \nu n_i(t) + I_\mathrm{bg} + \epsilon_i(t),
\end{align}

where $I_\mathrm{bg}$ is the background fluorescent intensity and $\epsilon_i(t)$ is noise in the measurement.  This noise can be due to shot noise, noise in the optics, errors in segmentation, etc.  Here, $n_i(t)$ is the number of fluorescent (non-photobleached) molecules in cell $i$ at time $t$.

We make measurements at time points $\{t_k\}$, with $k = 0, 1, \ldots K$.  For notational convenience, we define $I_{i,k} = I_i(t_k)$ and $n_{ik} = n_i(t_k)$.  Let the set of parameters $\boldsymbol{\sigma}$ characterize the noise $\epsilon_i(t)$.  We assume that $\boldsymbol{\sigma}$ is the same for all cells. Let the set of parameters $\boldsymbol{\tau}$ characterize the probability distribution $P(n_{i,k}\mid n_{i,k-1}, \boldsymbol{\tau}, \Delta t)$.  Again, we assume that $\boldsymbol{\tau}$ is the same for all cells.  So, our physical  parameters are $\nu, I_\mathrm{bg}, \boldsymbol{\sigma}, \boldsymbol{\tau}$.  We measure $\{I_{ik}, t_k\}$.  We have the $n_{i,k}$ as unknown parameters.

### Hierarchical Bayesian model

By Bayes's theorem,

\begin{align}
P(\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma}, \boldsymbol{\tau} \mid \{I_{ik}\}, \{t_k\}) &\propto 
P(\{I_{ik}\} \mid\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma}, \boldsymbol{\tau}, \{t_k\}) \\[1em]
&\;\;\;\;\times
P(\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma}, \boldsymbol{\tau}, \{t_k\}).
\end{align}

Now, the fluorescence intensity depends explicitly only on $\nu$, $n_{ik}$, $I_\mathrm{bg}$, and $\boldsymbol{\sigma}$.  So, the likelihood is

\begin{align}
P(\{I_{ik}\} \mid\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma}, \boldsymbol{\tau}, \{t_k\}) = P(\{I_{ik}\} \mid\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma})
\end{align}

We assume that the physical parameters are all independent of time and of each other. Then, we can write the prior as

\begin{align}
P(\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma}, \boldsymbol{\tau}, \{t_k\}) = P(\{n_{ik}\}, \boldsymbol{\tau}, \{t_k\}) \,P(\nu)\,P(I_\mathrm{bg}) \,P(\boldsymbol{\sigma})
\end{align}

To specify the probability $P(\{n_{ik}\}, \boldsymbol{\tau}, \{t_k\})$, we will assume that photobleaching is a memoryless process.  Then, we can write

\begin{align}
P(\{n_{ik}\}, \boldsymbol{\tau}, \{t_k\}) = \prod_{k=1}^K P(n_{ik}, n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1}).
\end{align}

Using the definition of conditional probability, this is

\begin{align}
P(\{n_{ik}\}, \boldsymbol{\tau}, \{t_k\}) &= \prod_{k=1}^K P(n_{ik} \mid n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1})\,P(\boldsymbol{\tau}, t_k - t_{k-1}) \\[1em]
&=\prod_{k=1}^K P(n_{ik} \mid n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1})\,P(\boldsymbol{\tau})\, P(t_k - t_{k-1}),
\end{align}

where we have used the fact that $\boldsymbol{\tau}$ is independent of time in the second line.  We know the time points essentially exactly, so $P(t_k-t_{k-1}) = \text{constant}$.  Then, we have

\begin{align}
P(\{n_{ik}\}, \boldsymbol{\tau}, \{t_k\}) \propto
\prod_{k=1}^K P(n_{ik} \mid n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1})\,P(\boldsymbol{\tau}).
\end{align}

Putting it all together, our updated posterior is

\begin{align}
P(\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma}, \boldsymbol{\tau} \mid \{I_{ik}\}, \{t_k\}) &\propto P(\{I_{ik}\} \mid\nu, \{n_{ik}\}, I_\mathrm{bg}, \boldsymbol{\sigma})\,\prod_{k=1}^K P(n_{ik} \mid n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1}) \\[1em]
&\;\;\;\;\times P(\nu)\,P(I_\mathrm{bg}) \,P(\boldsymbol{\sigma})\,P(\boldsymbol{\tau}).
\end{align}

Note that implicit in our prior is the fact that

\begin{align}
0 \le n_{iK} \le n_{i,K-1} \le \cdots \le n_{i,1} \le n_{i,0}
\end{align}

for all $i$.

Up until this point, we have done minimal physical modeling, except for assuming independence of parameters and memorylessness of photobleaching.  To proceed, we need to specify our model.  That is, we must specify the probabilities.  We are free to choose reasonable models.

### Specification of photobleaching probability

To specify the probability $P(n_{ik} \mid n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1})$, we consider a single fluorescent fluorophore.  Let $p(\boldsymbol{\tau}, t_k-t_{k-1})$ be the probability that it photobleaches in the time interval $(t_{k-1}, t_k]$.  If all fluorophores in a given cell are identical, then the number of them that will survive photobleaching in this time interval, $n_{ik}$, is binomially distributed.  That is,

\begin{align}
P(n_{ik} \mid n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1}) = \frac{n_{i,k-1}!}{n_{ik}!(n_{i,k-1} - n_{ik})!}\,p^{n_{i,k-1} - n_{ik}}(1-p)^{n_{ik}}.
\end{align}

The modeling decision, then, in in the specification of $p(\boldsymbol{\tau}, t_k-t_{k-1})$.  If photobleaching is a Poisson process, the waiting time to photobleach is exponentially distributed.  In that case,

\begin{align}
p(\tau, t_k-t_{k-1}) = \int_{0}^{t_k-t_{k-1}}\mathrm{d}t\, \frac{\mathrm{e}^{-t/\tau}}{\tau}
= 1 - \mathrm{e}^{-(t_k-t_{k-1})/\tau}.
\end{align}

If photobleaching requires two Poisson processes to happen in succession, we have

\begin{align}
p(\tau_1, \tau_2, t_k, t_{k-1}) = \int_{0}^{t_k-t_{k-1}}\mathrm{d}t\, \frac{\mathrm{e}^{-t/\tau_1} - \mathrm{e}^{-t/\tau_2}}{\tau_1-\tau_2}
= 1 - \frac{\tau_1 \mathrm{e}^{-t/\tau_1} - \tau_2 \mathrm{e}^{-t/\tau_2}}{\tau_1 - \tau_2}. \\[1em]
\end{align}

### Gaussian likelihood

For purposes of discussion here, we will assume that $\epsilon_i(t)$ does not depend on $t$ or the magnitude of the fluorescence signal, but is Gaussian distributed noise with zero mean and cell-and-time independent variance $\sigma^2$.   We might consider intensity-dependent noise if we suspect that to be the case with our optics/analysis. We assume further that the measurement of each cell is independent of all others.  Finally, we assume that the measurement at each time point is also independent of other time points.  Then, we have

\begin{align}
P(\{I_{ik}\} \mid \nu, \{n_{ik}\}, I_\mathrm{bg}, \sigma)
= \prod_i \prod_k \frac{1}{\sqrt{2\pi \sigma^2}}
\,\exp\left[-\frac{1}{2\sigma^2}\left(I_{ik} - I_\mathrm{bg} - \nu n_{ik}\right)^2\right].
\end{align}

## Sample data

We will generate sample data by simulating the photobleaching process.  We will consider photobleaching as a Poisson process with time scale $\tau = 10$.  We will take $\nu = 10$, $\sigma = 10$, and $I_\mathrm{bg} = 70$.  We will sample every one time unit, and we will take $n_{i0}$ drawn from a Gaussian distribution with mean 100 and variance 10.  We will consider a field of 50 cells and measure them for 60 time units.  All units are arbitrary but consistent.

In [6]:
def photobleach_cell(n, delta_t, tau):
    """Return number of surviving fluorphores."""
    return n - np.random.binomial(n, 1-np.exp(-delta_t/tau))

def measure_cell(nu, n, I_bg, sigma):
    """Return measured fluorescence intensity of a cell."""
    return max(0, I_bg + nu*n + np.random.normal(0, sigma))

# Specify parameters
t = np.arange(60)
I_bg = 70
sigma = 10
nu = 10
tau = 10
n_cells = 50
std_initial_copy_number = 10
mean_initial_copy_number = 100

# Compute initial counts
n0 = np.random.normal(mean_initial_copy_number, std_initial_copy_number, 
                     size=n_cells).astype(int)
n0[n0<0] = 0

# Time intervals
dt = np.diff(t)

# Set initial counts/measurements
I = np.empty((n_cells, len(t)))
n = np.empty_like(I, dtype=int)
n[:,0] = n0
I[:,0] = np.array([measure_cell(nu, n_, I_bg, sigma) for n_ in n0])

# Fill out counts
for i in range(1, len(t)):
    n[:,i] = np.array([photobleach_cell(n_, dt[i-1], tau) for n_ in n[:,i-1]])
    I[:,i] = np.array([measure_cell(nu, n_, I_bg, sigma) for n_ in n[:,i]])

To get an idea of what this looks like, we'll plot a the photobleaching of the cells and also the mean intensities of all cells.

In [8]:
p = bp.figure(height=300, x_axis_label='t (a.u.)', y_axis_label='I (a.u.)')
[p.line(t, I_vals, line_width=1, alpha=0.5) for I_vals in I]
p.line(t, I.mean(axis=0), color='tomato', line_width=4)
bi.show(p)

## MCMC set-up

Now that we have some sample data to play with, we can set up an MCMC calculation.

### Approximating the binomial distribution

In order to use speedy continuous-variable samplers, we will approximate the discrete variables $n_{ik}$ as continuous.  We will therefore approximate the binomial distribution as

\begin{align}
P(n_{ik} \mid n_{i,k-1}, \boldsymbol{\tau}, t_k - t_{k-1}) = c\,\frac{\Gamma(n_{i,k-1}+1)}{\Gamma(n_{ik}+1)\Gamma(n_{i,k-1} - n_{ik}+1)}\,p^{n_{i,k-1} - n_{ik}}(1-p)^{n_{ik}},
\end{align}

where $c$ is a normalization constant.  This constant is in general a function of $p$ and $n_{i,k-1}$, but if $n_{i,k-1}$ is not too small and $p$ is not too close to zero or one, $c$ is very close to unity and does not dependend appreciably on $p$ and $n_{i,k-1}$.  We will go ahead with this, but to be careful, we should use a sampler that considers $n_{ik}$ as discrete and uses the exact binomial distribution.

### Organization of parameters

We will organize our parameters as a NumPy array as follows.  The first four parameters are the physical parameters, $\nu$, $I_\mathrm{bg}$, $\sigma$, and $\tau$.  After that come the $n_{ik}$ values.  If `n` is a 2D array where each row corresponds to a cell and each column to a time point, then the $n_{ik}$ values appear in the array of parameters as `n.flatten()`.

### The log likelihood

The likelihood is just the product of Gaussians.

In [ ]:
def log_like(p, I):
    """Log likelihood"""
    nu, I_bg, sigma, tau = p[:4]
    n = p[4:]
    
    return -len(n) * np.log(sigma) + \
                np.sum((I - I_bg - nu * n)**2) / 2 / sigma**2
    
def log_prior(p, I, t):
    """Log prior"""
    nu, I_bg, sigma, tau = p[:4]
    n = p[4:].reshape((len(t)))

Now, we are left with modeling the photobleaching process; that is specifying $P(\{n_{ik}\} \mid \{N_i\}, \boldsymbol{\tau}, \{t_{ik}\})$.  We define $\theta_{ijk}$ to be unity if molecule $j$ of cell $i$ is fluorescent at time $k$.  We then have

\begin{align}
n_{ik} = \sum_{j=1}^{N_i} \theta_{ijk}.
\end{align}



\begin{align}
P(\{n_{ik}\} \mid \{N_i\}, \boldsymbol{\tau}, \{t_{ik}\}) = \prod_i \prod_k \frac{1}{\tau}
\end{align}

In [7]:
def binom_approx_cdf_single_value(x, n, p):
    """
    Approximate binomial distribution
    """
    if x <= 0:
        return 0    
    elif x > n+1:
        return 1
    
    return 1 - scipy.special.betainc(x, n+1-x, p)

def binom_approx_cdf(x, n, p):
    return np.array([binom_approx_cdf_single_value(a, n, p) for a in x])

def binom_approx(x, n, p):
    log_P = scipy.special.gammaln(n+1) - scipy.special.gammaln(n-x+1) - \
            scipy.special.gammaln(x+1) + x * np.log(p) + (n-x) * np.log(1-p)
    return np.exp(log_P)

In [18]:
def cheb_points_1d(n, xends=[-1, 1]):
    """
    Computes Chebyshev points for a 1-D Chebyshev grid, returned as ndarray
    """
    
    x = np.cos(np.pi * np.arange(n) / (n-1))

    if xends != [-1.0, 1.0]:
        x = (xends[1] - xends[0]) / 2.0 * x + (xends[1] + xends[0]) / 2.0

    return x

def compute_cheb_a(a, m, n):
    """
    compute_cheb_a(a, m, n)
    
    Recursion formula for computing coefficients of the ell'th
    derivative for Chebychev differentiation by FFT.  This takes a lot
    of time, so we're weaving it.
    """

    for ell in range(1,m+1):
        a[n-ell-1, ell] = 2.0 * (n-ell) * a[n-ell, ell-1]
        for k in range(n-ell-2, 0, -1):
            a[k, ell] = a[k+2, ell] + 2.0 * (k + 1) * a[k+1, ell-1]
        a[0, ell] = a[1, ell-1] + a[2, ell] / 2.0
    return a


def diff_cheb_fft_1d(f, m, L=None, real_data=True, **kwargs):
    """
    Computes the mth derivative of the 1-D function f defined at
    Chebyshev grid points on an interval of length L.  The standard
    length of the interval is 2, corresponding to Chebyshev points
    between -1 and 1.  Uses FFTs to compute the derivatives.  

    NOTE: This calculation can be speeded up by a factor of two using DCTs,
    available in the next distribution of SciPy.

    Adapted from Matlab code by J.A.C. Weideman and S.C. Reddy, 2000.
    """
    
    n = len(f)

    # Extend and compute FFT, a0 contains Chebyshev coeffs of f
    a0 = np.fft.fft(np.concatenate((f, np.flipud(f[1:n-1]))))
    a0 = a0[0:n] * np.concatenate(((0.5,), np.ones(n-2), (0.5,))) / (n-1) 
    
    # Recursion formula for computing coefficients of the ell'th derivative
    a = np.column_stack((a0, np.zeros((n, m))))
    a = compute_cheb_a(a, m, n)

    # Transform back to physical space
    back = np.concatenate(
        ((2.0*a[0, m],), a[1:n-1, m], (2.0*a[n-1, m],), np.flipud(a[1:n-1, m])))

    Dmf = 0.5 * np.fft.fft(back)
    Dmf = Dmf[0:n]

    # Real data in, real derivative out
    if real_data:
        Dmf = np.real(Dmf)

    # Adjust for specified interval length
    if L is not None:
        Dmf = Dmf * (2.0 / L)**m

    return Dmf

In [40]:
n = 25
p = 0.7
x = np.arange(0, n+1)
x_cont = cheb_points_1d(32, xends=[1/2, n+1/2])
cont = binom_approx_cdf(x_cont, n, p)
cont_deriv = diff_cheb_fft_1d(cont, 1, L=n)
disc = st.binom.pmf(np.arange(0, n+1), n, p)

p = bp.figure(height=300)
p.circle(x, disc, size=7)
# p.line(x_cont[1:], np.diff(cont), line_width=2, line_join='bevel')
p.line(x_cont-1/2, cont_deriv, line_width=2, line_join='bevel', color='darkgray')
bi.show(p)


In [11]:
n = 10
p = 0.04
x = np.arange(0, n+1)
x_cont = np.linspace(0, n+1, 300)
disc = st.binom.cdf(np.arange(0, n+1), n, p)
cont = binom_approx_cdf(x_cont, n, p)

p = bp.figure()
p.circle(x, disc, size=7)
# p.line(x_cont[1:], np.diff(cont), line_width=2, line_join='bevel')
p.line(x_cont, cont, line_width=2, line_join='bevel', color='darkgray')
bi.show(p)

In [129]:
n = 5
p_vals = np.linspace(0, 1, 1000)
x = np.arange(0, n+1)
x_cont = np.linspace(0, n+1, 3000)

norm = np.array([np.trapz(binom_approx(x_cont, n, p), x=x_cont) for p in p_vals])


p = bp.figure(y_axis_type='log')
p.line(p_vals, norm, line_width=2, line_join='bevel')
bi.show(p)

In [22]:
np.trapz(cont, x=x_cont)

3.8903140543637882

In [11]:
disc = st.binom.pmf(np.arange(0, n+1), n, p)

array([  1.68703194e-002,   7.02929973e-002,   1.44979307e-001,
         1.97332946e-001,   1.99388497e-001,   1.59510798e-001,
         1.05232818e-001,   5.88802672e-002,   2.85201294e-002,
         1.21474625e-002,   4.60591288e-003,   1.57019757e-003,
         4.85234666e-004,   1.36861060e-004,   3.54372387e-005,
         8.46556257e-006,   1.87388755e-006,   3.85800378e-007,
         7.41236836e-008,   1.33292589e-008,   2.24931244e-009,
         3.57033721e-010,   5.34198180e-011,   7.54845255e-012,
         1.00908133e-012,   1.27816969e-013,   1.53626164e-014,
         1.75437286e-015,   1.90579195e-016,   1.97150891e-017,
         1.94412684e-018,   1.82915160e-019,   1.64337839e-020,
         1.41098145e-021,   1.15852643e-022,   9.10270768e-024,
         6.84810185e-025,   4.93556890e-026,   3.40943904e-027,
         2.25838911e-028,   1.43501808e-029,   8.75011024e-031,
         5.12159230e-032,   2.87841428e-033,   1.55368953e-034,
         8.05616791e-036,   4.01348945e-